In [5]:
from itertools import count
from tqdm import tqdm
%run env_test.ipynb
%run agent.ipynb

In [12]:
def train(num_episodes, env, agent, vis=False):
    steps_per_e = np.zeros(num_episodes)
    for episode in tqdm(range(num_episodes)):
        # choose s0
        s = env.reset()
        agent.store_s(s, t=0)
        
        # choose a0
        a = agent.choose_action(s)
        agent.store_a(s, a, t=0)
        
        # T <- inf
        T = np.inf
        for t in count():
            if t >= 100_000:
                print("too long")
                break
            if t < T:
                # take action a(t)
                a = agent.a_t[t % (agent.n + 1)]

                # observe and store r and sp
                sp, r, terminal = env.step(a)
                agent.store_s(sp, t+1)
                agent.store_r(r, t+1)

                if terminal:
                    T = t + 1
                else:
                    # select and store a(t+1)
                    ap = agent.choose_action(sp)
                    agent.store_a(sp, ap, t=t + 1)
            
            tau = t - agent.n + 1
            if tau >= 0:
                agent.learn(tau, T)
                
            if tau == T - 1:
                break

            if vis:
                cont = env.render()
                if not cont:
                    pg.quit()
                    vis = False

        steps_per_e[episode] = t
        r_per_e[episode] = r_t

    try:
        pg.quit()
    except:
        pass

    return Q, steps_per_e

def train(num_episodes, env, agent, vis=False):
    steps_per_e = np.zeros(num_episodes)
    r_per_e = np.zeros(num_episodes)
    for episode in tqdm(range(num_episodes)):
        terminal = False
        s = env.reset()
        r_t = 0
        t = 0
        while not terminal:
            t += 1
            a = agent.choose_action(s)
            sp, r, terminal = env.step(a)
            Q, s_count, sa_count = agent.learn(s, a, r, sp)
            r_t += r
            s = sp

            if vis:
                cont = env.render()
                if not cont:
                    pg.quit()
                    vis = False

        steps_per_e[episode] = t
        r_per_e[episode] = r_t

    try:
        pg.quit()
    except:
        pass

    return Q, s_count, sa_count, steps_per_e, r_per_e

def play(env, Q):
    pg.init()
    agent = AgentSmall(n_actions, n_dims, gamma, alpha_p, Q)
    terminal = False
    s = env.reset()
    while True:
        a = agent.choose_action_egreedy(s)
        sp, r, terminal = env.step(a)
        if terminal:
            pg.quit()
            return
        cont = env.render()
        if cont is False:
            return


In [13]:
if __name__ == "__main__":
    actions = np.array([[0, 1], [-1, 0], [0, -1], [1, 0]], dtype=int)
    n_actions = actions.shape[0]
    n_dims = (n_actions, H, W)
    gamma = 0.99
    alpha_p = 0.99
    
    n_states = 2
    
    env = TestEnv(n_dims, n_states, actions)
    num_episodes = 1000


In [14]:
agent = Agent( 
                 n_actions=n_actions, 
                 n_dims=n_dims, 
                 gamma=gamma, 
                 alpha_p=alpha_p)    
Q, steps_per_e = train(num_episodes=num_episodes, env=env, agent=agent, vis=True)

np.save(f"Q_s_{num_episodes + offset}", Q)
np.save(f"s_count_s_{num_episodes + offset}", s_count)
np.save(f"sa_count_s_{num_episodes + offset}", sa_count)

plt.scatter(range(num_episodes), steps_per_e)
plt.scatter(range(num_episodes), r_per_e)

  0%|          | 0/1000 [00:00<?, ?it/s]


IndexError: index 2 is out of bounds for axis 0 with size 2